In [1]:
import numpy as np
import psycopg2
import pandas as pd
import datetime as dt
from dateutil.relativedelta import *
from sqlalchemy import create_engine, func, inspect

### Create Database Engine

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# SQL Query 1: Return Total Precipiation by Date

In [3]:
query_m_1 = """
                SELECT 
                    date,
                    sum(prcp) as Total_Precipitation
                FROM 
                    measurement
                GROUP BY
                    date
                ORDER BY
                    date ASC;
            """

In [4]:
conn = engine.connect()
df = pd.read_sql(query_m_1, conn)
conn.close()
df.head()

,date,Total_Precipitation
0,2010-01-01,1.06
1,2010-01-02,0.02
2,2010-01-03,0.00
3,2010-01-04,0.01
4,2010-01-05,0.03


# SQL Query 2: Return All Stations

In [5]:
query_m_2 = """
                SELECT 
                    station as station_ID,
                    name as station_name,
                    latitude,
                    longitude,
                    elevation
                FROM 
                    station
                ORDER BY
                    station_ID ASC;
            """

In [6]:
conn = engine.connect()
df = pd.read_sql(query_m_2, conn)
conn.close()
df.head()

,station_ID,station_name,latitude,longitude,elevation
0,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.3152,-157.9992,0.9
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4
4,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9


# SQL Query 3:  Return Temperature for the Previous Year from the Most Active Station

### Query with Hard-Coded Date

In [7]:
query_m_3 = """
                SELECT 
                    m.id as Measurement_ID,
                    m.station as Station_ID,
                    s.name as Station_Name,
                    m.date as Date,
                    m.tobs as Temperature
                FROM 
                    "measurement" m
                    JOIN "station" s on m.station = s.station
                WHERE
                    m.station = 'USC00519281'
                    AND date > '2016-08-23'
                ORDER BY
                    date DESC;
            """

In [8]:
conn = engine.connect()
df = pd.read_sql(query_m_3, conn)
conn.close()
df.head()

,Measurement_ID,Station_ID,Station_Name,Date,Temperature
0,14959,USC00519281,"WAIHEE 837.5, HI US",2017-08-18,79.0
1,14958,USC00519281,"WAIHEE 837.5, HI US",2017-08-17,76.0
2,14957,USC00519281,"WAIHEE 837.5, HI US",2017-08-16,76.0
3,14956,USC00519281,"WAIHEE 837.5, HI US",2017-08-15,77.0
4,14955,USC00519281,"WAIHEE 837.5, HI US",2017-08-14,77.0


### Query using Relative Date

In [9]:
query_m_99 = f"""
                SELECT
                    m.id as Measurement_ID,
                    m.station as Station_ID,
                    s.name as Station_Name,
                    m.date as Date,
                    m.tobs as Temperature
                FROM 
                    "measurement" m
                    JOIN "station" s on m.station = s.station
                WHERE
                    m.station = 'USC00519281'
                ORDER BY
                    date DESC;
            """

In [10]:
#Import SQL Query Results into DataFrame
conn = engine.connect()
df = pd.read_sql(query_m_99, conn)
conn.close()

df = pd.DataFrame(df)
# Convert Date Column to Timestamp
df['Date'] = pd.to_datetime(df['Date'])
#Set Date as Index
df.set_index('Date', inplace=True)
#jsonified dataframe does not return index
#creating duplicate date column
df['Date_1'] = df.index
df['Date_1'] = df['Date_1'].astype(str)

#Filter for relative 12 months
start_date = df.index.max()
end_date = df.index.max() - relativedelta(months=12)
mask = (df.index > end_date) & (df.index <= start_date)
last12_df = df.loc[mask]
last12_df.head()

,Measurement_ID,Station_ID,Station_Name,Temperature,Date_1
Date,,,,,
2017-08-18,14959,USC00519281,"WAIHEE 837.5, HI US",79.0,2017-08-18
2017-08-17,14958,USC00519281,"WAIHEE 837.5, HI US",76.0,2017-08-17
2017-08-16,14957,USC00519281,"WAIHEE 837.5, HI US",76.0,2017-08-16
2017-08-15,14956,USC00519281,"WAIHEE 837.5, HI US",77.0,2017-08-15
2017-08-14,14955,USC00519281,"WAIHEE 837.5, HI US",77.0,2017-08-14


In [11]:
last12_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 356 entries, 2017-08-18 to 2016-08-19
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Measurement_ID  356 non-null    int64  
 1   Station_ID      356 non-null    object 
 2   Station_Name    356 non-null    object 
 3   Temperature     356 non-null    float64
 4   Date_1          356 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 16.7+ KB


# SQL Query 4: Min, Max & Average Temperatures for a Given Start Date

In [12]:
date = '2016-08-23'
query_m_4 = f"""
                SELECT 
                    date,
                    min(tobs) as min_temp,
                    max(tobs) as max_temp,
                    avg(tobs) as avg_temp
                FROM 
                    measurement
                WHERE
                    date = '{date}';
            """

In [13]:
conn = engine.connect()
df = pd.read_sql(query_m_4, conn)
conn.close()
df.head()

,date,min_temp,max_temp,avg_temp
0,2016-08-23,74.0,81.0,78.285714


# SQL Query 5: Min, Max & Average Temperatures for Date Range

In [14]:
start = '2016-08-23'
end = '2017-08-23'
query_m_5 = f"""
                SELECT 
                    min(tobs) as min_temp,
                    max(tobs) as max_temp,
                    avg(tobs) as avg_temp
                FROM 
                    measurement
                WHERE
                    date >= '{start}'
                    AND date < '{end}';
            """

In [15]:
conn = engine.connect()
df = pd.read_sql(query_m_5, conn)
conn.close()
df.head()

,min_temp,max_temp,avg_temp
0,58.0,87.0,74.580413
